In [1]:
import sys

sys.path.append("../")

DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
import itertools as it
import backtrader as bt
import requests
import empyrical
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

In [3]:
def zscore(x: pd.Series, window: int):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

In [4]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'

LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=1000):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, interval='1m'):
    batches = []

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= datetime.utcnow():
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        df = df.astype('float32')
        return df



In [5]:
def print_analysis(analyzer):
    """
    Function to print the Technical Analysis results in a nice format.
    """
    # Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total, 2)
    strike_rate = (total_won / total_closed) * 100
    # Designate the rows
    h1 = ["Total Open", "Total Closed", "Total Won", "Total Lost"]
    h2 = ["Strike Rate", "Win Streak", "Losing Streak", "PnL Net"]
    r1 = [total_open, total_closed, total_won, total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    # Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    # Print the rows
    print_list = [h1, r1, h2, r2]
    row_format = "{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format("", *row))

In [6]:
data = {}

# config

In [7]:

timeframe = '1h'
stationary_df = pd.read_csv(f'stationary_df_{timeframe}.csv')
stationary_df

,i,pairs,half_life,hedge_ratio,sid_1,sid_2
0,26,['ALGO' 'UNI'],43,[ 21.54100759 -35.86843553],ALGO,UNI
1,12,['ETC' 'NEAR'],62,[23.24951095 -9.37038132],ETC,NEAR
2,16,['LINK' 'ADA'],72,[ 24.88430038 -12.526589 ],LINK,ADA
3,36,['NEAR' 'MANA'],77,[ 18.13154745 -13.2253253 ],NEAR,MANA
4,18,['LINK' 'DOT'],81,[ 25.0633453 -13.42932245],LINK,DOT
5,13,['ETC' 'MANA'],81,[19.15963163 -5.78117205],ETC,MANA
6,7,['ETC' 'ALGO'],83,[16.70419245 -4.95667008],ETC,ALGO
7,11,['ETC' 'XMR'],85,[ 16.36272503 -15.12756944],ETC,XMR
8,1,['LTC' 'ADA'],97,[17.88059467 -6.75741228],LTC,ADA
9,8,['ETC' 'DOT'],97,[14.65877894 -3.16906095],ETC,DOT


In [8]:
# # Get the dates from the args
# today = datetime.now()
# if timeframe == '1h':
#     fromdate = today - timedelta(days=4 * 30)
#     start_trading_at = today - timedelta(days=30)
# else:
#     fromdate = today - timedelta(days=20)
#     start_trading_at = today - timedelta(days=20)

# nb_symbols = len(hedge_ratio)
# pair = [selected_row[f'sid_{i + 1}'] for i in range(nb_symbols)]
# print(pair)
# position_size = 5

# for symbol in pair:
#     df_ = get_candles(base=symbol, quote='USDT', start_date=fromdate, interval=timeframe)
#     data[symbol] = df_

# start_trading_at

In [9]:

def convert_hedge_ratio(hedge_ratio):
    hedge_ratio = hedge_ratio.replace('[', '')
    hedge_ratio = hedge_ratio.replace(']', '')
    hedge_ratio = hedge_ratio.strip()
    hedge_ratio = [float(x) for x in hedge_ratio.split(' ') if x != '']
    return hedge_ratio


# Get the dates from the args
today = datetime.now()
if timeframe == '1h':
    fromdate = today - timedelta(days=4 * 30)
    start_trading_at = today - timedelta(days=60)
elif timeframe in ['15m']:
    fromdate = today - timedelta(days=30)
    start_trading_at = today - timedelta(days=30)
elif timeframe in ['1d']:
    fromdate = today - timedelta(days=6*30)
    start_trading_at = today - timedelta(days=252)
else:
    fromdate = today - timedelta(days=45)
    start_trading_at = today - timedelta(days=30)

# fromdate = datetime(2023, 1, 2) - timedelta(days=30)
# start_trading_at = datetime(2023, 1, 2) - timedelta(days=30)


In [10]:
class Strategy(bt.Strategy):
    params = dict(
        symbols=[],
        hedge_ratio=[],
        half_life=None,
        nb_symbols=2,
        sl_upper_limit=10,
        middle=0,
        upper_limit=3,
        upper_limit2=5,
        lower_limit=-3,
        lower_limit2=-5,
        sl_lower_limit=-10,
        position_size=10,
        start_trading_at=None,
        dca_enabled=True,
        fixed_weight=True,
        debug=False,
        enable_tp_sl=False,
        tp_dollar=3,
        sl_dollar=20,
    )

    def __init__(self) -> None:
        super().__init__()
        self.portfolio = {d._name: d for d in self.datas if d._name in self.p.symbols}
        self.lookback = self.p.half_life
        self.status = 0
        self.in_position = False
        self.dca_count = 0
        self.hedge_ratio = self.p.hedge_ratio
        self.half_life = self.p.half_life
        self.fixed_weight = self.p.fixed_weight
        self.next_dca_level = None
        self.tp_dollar = self.p.tp_dollar
        self.sl_dollar = self.p.sl_dollar
        self.log(
            f"Start trade with {self.p.symbols}, {self.p.hedge_ratio}, {self.p.half_life}, {self.p.nb_symbols}"
        )

    def log(self, txt, dt=None):
        dt = dt or self.data.datetime[0]
        dt = bt.num2date(dt)
        if self.p.debug:
            print("%s, %s" % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = (
                    f"BUY COMPLETE {order.data._name}, "
                    f"price: {order.executed.price:.4f}, "
                    f"size: {order.executed.size}, "
                    f"cost {order.executed.price * order.executed.size:.4f}"
                )
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = (
                    f"SELL COMPLETE {order.data._name}, "
                    f"price: {order.executed.price:.4f}, "
                    f"size: {order.executed.size}"
                    f"cost {order.executed.price * order.executed.size:.4f}"
                )
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log("Order cancelled %s ," % order.Status[order.status])
            pass  # Simply log

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log("OPERATION PROFIT, GROSS %.4f, NET %.4f" % (trade.pnl, trade.pnlcomm))
        self.log(f"ACCOUNT VALUE: {self.broker.get_value()}")

    def get_price_series(self, symbol):
        d = self.portfolio[symbol]
        lookback = self.lookback * 3
        close = d.close.get(size=lookback)
        t = [bt.num2date(dt) for dt in d.datetime.get(size=lookback)]
        s = pd.Series(np.log(close), index=t)

        return s

    def calculate_spread(self):
        spread = None
        for i in range(self.p.nb_symbols):
            close_s = self.get_price_series(self.p.symbols[i])
            if spread is None:
                spread = close_s * self.hedge_ratio[i]
            else:
                spread += close_s * self.hedge_ratio[i]

        spread.dropna(inplace=True)
        return spread

    def calculate_zscore(self, spread):
        zscore_df = zscore(spread, self.lookback)
        return zscore_df

    def long_spread(self, position_size=None):
        if position_size is None:
            position_size = self.p.position_size

        for i, symbol in enumerate(self.p.symbols):
            if not self.fixed_weight:
                size = np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )
                size = (self.hedge_ratio[i] * position_size) / self.portfolio[
                    symbol
                ].close[0]
            else:
                size = np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )

            if size > 0:
                self.log(
                    f"Buy {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.buy(data=self.portfolio[symbol], size=abs(size))
            else:
                self.log(
                    f"Sell {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.sell(data=self.portfolio[symbol], size=abs(size))

    def short_spread(self, position_size=None):
        if position_size is None:
            position_size = self.p.position_size

        for i, symbol in enumerate(self.p.symbols):
            if not self.fixed_weight:
                size = -np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )
                size = (
                    -(self.hedge_ratio[i] * position_size)
                    / self.portfolio[symbol].close[0]
                )
            else:
                size = -np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )
            if size > 0:
                self.log(
                    f"Buy {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.buy(data=self.portfolio[symbol], size=abs(size))
            else:
                self.log(
                    f"Sell {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.sell(data=self.portfolio[symbol], size=abs(size))

    def close_all(self):
        for i, symbol in enumerate(self.p.symbols):
            self.close(self.portfolio[symbol])
        self.status = 0
        self.in_position = False
        self.dca_count = 0

    def unrealized_pnl(self):
        unrealizel_pnl = 0
        for symbol in self.p.symbols:
            pos = self.getposition(self.portfolio[symbol])
            pnl = (self.portfolio[symbol].close[0] - pos.price) * pos.size
            unrealizel_pnl += pnl
        return unrealizel_pnl

    def next(self):
        dt = self.data.datetime[0]
        dt = bt.num2date(dt)
        if self.p.start_trading_at is not None and dt <= self.p.start_trading_at:
            return

        spread = self.calculate_spread()
        if len(spread) < self.lookback * 2:
            return

        # if adf_test(spread, verbose=False) > 0.05 and not self.in_position:
        #     self.log(f'Spread is not stationary')
        # #     # if not self.calculate_hedge_ratio():
        # #     #     return
        #     return

        if self.in_position and self.p.enable_tp_sl:
            if self.unrealized_pnl() > self.tp_dollar:
                self.close_all()
                self.log(f"Take profit unrealized pnl > {self.tp_dollar}")
                return
            elif self.unrealized_pnl() <= -self.sl_dollar:
                self.close_all()
                self.log(f"Stop loss unrealized pnl < -{self.sl_dollar}")
                return

        zscore = self.calculate_zscore(spread).iloc[-1]
        std = spread.rolling(self.lookback).std().iloc[-1]
        if (
            (zscore >= self.p.upper_limit)
            and (self.status != 1)
            and not self.in_position
        ):
            self.log(
                f"Short spread zscore {zscore} > {self.p.upper_limit}, {self.data0.close[0]} - {self.data1.close[0]}"
            )
            self.short_spread()
            self.status = 1  # The current status is "short the spread"
            self.in_position = True
            self.next_dca_level = spread.iloc[-1] + 2 * std
        elif (
            self.next_dca_level is not None
            and spread.iloc[-1] >= self.next_dca_level
            and (self.status == 1)
            and self.in_position
            and self.dca_count == 0
            and self.p.dca_enabled
        ):
            self.log(
                f"DCA {self.dca_count} Short spread zscore {spread.iloc[-1]} > {self.next_dca_level}"
            )
            self.short_spread(position_size=self.p.position_size * 1.5)
            self.dca_count += 1
            self.next_dca_level = spread.iloc[-1] + 2 * std
        elif (
            self.next_dca_level is not None
            and spread.iloc[-1] >= self.next_dca_level
            and (self.status == 1)
            and self.in_position
            and self.dca_count == 1
            and self.p.dca_enabled
        ):
            self.log(
                f"DCA {self.dca_count} Short spread zscore {spread.iloc[-1]} > {self.next_dca_level}"
            )
            self.short_spread(position_size=self.p.position_size * 2)
            self.dca_count += 1
            self.next_dca_level = spread.iloc[-1] + 2 * std
        elif (
            zscore < self.p.lower_limit and (self.status != 2) and not self.in_position
        ):
            self.log(
                f"Long spread zscore {zscore} > {self.p.lower_limit}, {self.data0.close[0]} - {self.data1.close[0]}"
            )
            self.long_spread()

            self.status = 2
            self.in_position = True
            self.next_dca_level = spread.iloc[-1] - 2 * std
        elif (
            self.next_dca_level is not None
            and spread.iloc[-1] <= self.next_dca_level
            and (self.status == 2)
            and self.in_position
            and self.dca_count == 0
            and self.p.dca_enabled
        ):
            self.log(
                f"DCA {self.dca_count} Long spread zscore {spread.iloc[-1]} < {self.next_dca_level}"
            )
            self.long_spread(position_size=self.p.position_size * 1.5)
            self.dca_count += 1
            self.next_dca_level = spread.iloc[-1] - 2 * std
        elif (
            self.next_dca_level is not None
            and spread.iloc[-1] <= self.next_dca_level
            and (self.status == 2)
            and self.in_position
            and self.dca_count == 1
            and self.p.dca_enabled
        ):
            self.log(
                f"DCA {self.dca_count} Long spread zscore {spread.iloc[-1]} < {self.next_dca_level}"
            )
            self.long_spread(position_size=self.p.position_size * 2)
            self.dca_count += 1
            self.next_dca_level = spread.iloc[-1] - 2 * std

        elif self.getposition(self.data0) and (
            (zscore <= self.p.middle and self.status == 1)
            or (zscore >= -self.p.middle and self.status == 2)
        ):
            self.log(f"Take profit {zscore} <> 0")
            self.close_all()

        elif self.getposition(self.data0) and (
            (zscore >= self.p.sl_upper_limit and self.status == 1)
            or (zscore <= self.p.sl_lower_limit and self.status == 2)
        ):
            self.log(f"Stop Loss {zscore} <> +- 2")
            self.close_all()


def run(
    data,
    pair,
    hedge_ratio,
    half_life,
    start_trading_at,
    nb_symbols,
    position_size,
    fixed_weight=False,
    debug=False,
    upper_limit=2.5,
    upper_limit2=4,
    middle=1,
    lower_limit=-2.5,
    lower_limit2=-4,
    enable_tp_sl=False,
    tp_dollar=3,
    sl_dollar=20,
):
    # Create a cerebro
    cerebro = bt.Cerebro()

    for symbol in pair:
        df_ = data[symbol]
        # Create the 1st data
        data0 = bt.feeds.PandasData(
            dataname=df_,
            plot=False,
        )

        # Add the 1st data to cerebro
        cerebro.adddata(data0, name=symbol)

    # Add the strategy
    cerebro.addstrategy(
        Strategy,
        symbols=pair,
        hedge_ratio=hedge_ratio,
        half_life=half_life,
        nb_symbols=nb_symbols,
        position_size=position_size,
        sl_upper_limit=14,
        upper_limit=upper_limit,
        upper_limit2=upper_limit2,
        middle=middle,
        lower_limit=lower_limit,
        lower_limit2=lower_limit2,
        sl_lower_limit=-14,
        dca_enabled=True,
        start_trading_at=start_trading_at,
        fixed_weight=fixed_weight,
        debug=debug,
        enable_tp_sl=enable_tp_sl,
        tp_dollar=tp_dollar,
        sl_dollar=sl_dollar,
    )

    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="return")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.DrawDown)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcash(1000)
    cerebro.broker.setcommission(commission=0.04 / 100, leverage=10)

    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
    # And run it
    results = cerebro.run()
    first_strat = results[0]
    print_analysis(first_strat.analyzers.ta.get_analysis())
    print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
    # Print out the final result
    print(
        f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%"
    )
    print(
        f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%"
    )
    return (
        round(results[0].analyzers.returns.get_analysis()["rnorm100"], 4),
        results[0].analyzers.drawdown.get_analysis()["max"]["drawdown"],
        results[0],
    )


In [13]:
import traceback

result = []
analyzers = {}


for i, row in stationary_df.head(10).iterrows():
    try:
        index = row.i

        half_life = stationary_df[stationary_df['i'] == index]['half_life'].iloc[-1]
        selected_row = stationary_df[stationary_df['i'] == index].iloc[-1]
        hedge_ratio = convert_hedge_ratio(selected_row['hedge_ratio'])
        print(selected_row, hedge_ratio, half_life)

        nb_symbols = len(hedge_ratio)
        pair = [selected_row[f'sid_{i + 1}'] for i in range(nb_symbols)]
        print(pair)
        
        fixed_weight = False
        position_size = int(200 / max(hedge_ratio))

        # fixed_weight = True
        # position_size = 500

        for symbol in pair:
            if symbol not in data:
                df_ = get_candles(base=symbol, quote='USDT', start_date=fromdate, interval=timeframe)
                data[symbol] = df_
            
        ret, dd, res = run(
            data, 
            pair, 
            hedge_ratio, 
            half_life, 
            start_trading_at, 
            nb_symbols, 
            position_size=position_size, 
            fixed_weight=fixed_weight,
            debug=False
        )
        
        result.append({
            'pair': pair,
            'hedge_ratio': hedge_ratio,
            'half_life': half_life,
            'ret': ret,
            'dd': dd,
        })
        analyzers[','.join(pair)] = res
        print('========================================================')
    except KeyboardInterrupt:
        break
    except:
        # traceback.print_exc()
        pass


i                                        8
pairs                       ['ALGO' 'UNI']
half_life                               42
hedge_ratio    [ 22.21408684 -36.27470728]
sid_1                                 ALGO
sid_2                                  UNI
Name: 0, dtype: object [22.21408684, -36.27470728] 42
['ALGO', 'UNI']
Starting Portfolio Value: 1000.00DT 1h 2023-04-18 08:00:00                     
Trade Analysis Results:
               Total Open     Total Closed   Total Won      Total Lost     
               0              64             35             29             
               Strike Rate    Win Streak     Losing Streak  PnL Net        
               54.6875        7              2              24.58          
Final Portfolio Value: 1024.58
Norm. Annual Return: 5.19%
Max Drawdown: 3.85%
i                                       13
pairs                       ['NEAR' 'UNI']
half_life                               47
hedge_ratio    [ 14.09766869 -33.51788637]
sid_1         

In [14]:

result_df = pd.DataFrame(result)
result_df['ret'] = result_df['ret'].astype('float32')
result_df.sort_values(by=['ret', 'dd'], ascending=[False, True], inplace=True)
result_df

,pair,hedge_ratio,half_life,ret,dd
8,"[ETC, SOL]","[21.03445934, -7.61099907]",73,12.8194,1.264360
5,"[ETC, NEAR]","[22.93635116, -11.22846431]",71,10.9131,2.111228
2,"[LINK, ADA]","[27.50261717, -15.28667834]",64,10.5135,1.630384
9,"[SOL, NEAR]","[12.4516065, -17.15567277]",80,7.1821,3.798783
3,"[LINK, DOT]","[31.17184576, -18.28933159]",69,7.1551,0.845124
7,"[NEAR, MANA]","[18.92096476, -12.99863007]",72,5.4852,1.022472
0,"[ALGO, UNI]","[22.21408684, -36.27470728]",42,5.1876,3.848025
1,"[NEAR, UNI]","[14.09766869, -33.51788637]",47,1.0674,4.733491
4,"[MANA, UNI]","[6.99129323, -24.91433234]",69,0.3440,8.731104
6,"[SOL, MANA]","[14.86970524, -13.808338]",71,0.2455,6.051541


In [11]:

index = 11

half_life = stationary_df[stationary_df['i'] == index]['half_life'].iloc[-1]
selected_row = stationary_df[stationary_df['i'] == index].iloc[-1]
hedge_ratio = convert_hedge_ratio(selected_row['hedge_ratio'])
print(selected_row, hedge_ratio, half_life)

nb_symbols = len(hedge_ratio)
pair = [selected_row[f'sid_{i + 1}'] for i in range(nb_symbols)]
print(pair)

fixed_weight = False
position_size = int(300 / max(hedge_ratio))

for symbol in pair:
    if symbol not in data:
        df_ = get_candles(base=symbol, quote='USDT', start_date=fromdate, interval=timeframe)
        data[symbol] = df_

ret, dd, res = run(
    data, 
    pair, 
    hedge_ratio, 
    half_life, 
    start_trading_at, 
    nb_symbols, 
    position_size=position_size, 
    fixed_weight=fixed_weight,
    debug=True,
    upper_limit=2.5,
    upper_limit2=5,
    middle=0.5,
    lower_limit=-2.5,
    lower_limit2=-5,
    enable_tp_sl=True,
    tp_dollar=3,
    sl_dollar=20
)

i                                       11
pairs                        ['ETC' 'XMR']
half_life                               85
hedge_ratio    [ 16.36272503 -15.12756944]
sid_1                                  ETC
sid_2                                  XMR
Name: 7, dtype: object [16.36272503, -15.12756944] 85
['ETC', 'XMR']
Starting Portfolio Value: 1000.00DT 1h 2023-04-29 08:00:00                    
2023-04-29T01:00:00, Start trade with ['ETC', 'XMR'], [16.36272503, -15.12756944], 85, 2
2023-02-28T19:00:00, Long spread zscore -2.5221697110789 > -2.5, 20.909000396728516 - 150.61000061035156
2023-02-28T19:00:00, Buy ETC, size = 14.086232959567159, cost = 294.52905054
2023-02-28T19:00:00, Sell XMR, size = -1.807955971160688, cost = 272.29624992
2023-02-28T20:00:00, BUY COMPLETE ETC, price: 20.9090, size: 14.086232959567159, cost 294.5291
2023-02-28T20:00:00, SELL COMPLETE XMR, price: 150.6300, size: -1.807955971160688cost -272.3324
2023-03-01T07:00:00, Take profit unrealized pnl > 3
20

# RESULT

In [15]:
# res = analyzers['BTC,UNI']

# # If no name has been specified, the name is the class name lowercased
# return_analyzer = res.analyzers.getbyname('return')
# # print(tret_analyzer.get_analysis())
# account_return = pd.Series(return_analyzer.get_analysis(), name='return')
# cum_r = (1 + account_return).cumprod()
# plt.axvline(x = start_trading_at, color = 'b', label = 'OOS')

# cum_r.plot()
# plt.show()
